# Binary Classification of Subjective/Objective tweets
## Features: 
- POS tags + tfidf count in BoW style

In [1]:
import pandas as p
import numpy as np
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk import pos_tag

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix as cm

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cross_validation import train_test_split

In [2]:
train = p.read_csv('./sub_obj_data/train_new.csv', usecols=(['class', 'text'])).dropna()
train = train.reindex(np.random.permutation(train.index))
train_large = p.read_csv('./sub_obj_data/training.csv', usecols=(['class', 'text'])).dropna()
train_large = train_large.reindex(np.random.permutation(train_large.index))

test  = p.read_csv('./sub_obj_data/test_ds.csv', usecols=(['class', 'text'])).dropna()

print len(train), len(test)

5946 278


# Preprocessor class
Helper class which tokenises tweets and includes pos tags.

In [3]:
from __future__ import unicode_literals, division
import re
import htmlentitydefs
import csv

from nltk.corpus import stopwords as sw
from nltk import pos_tag


class Preprocessor():

    def __init__(self):
        self.stopwords = list(sw.words('english'))
        self.word_re = word_re
        self.emoticon_re = emoticon_re
        self.html_entity_digit_re = html_entity_digit_re
        self.html_entity_alpha_re = html_entity_alpha_re
        self.amp = amp
        self.punct_re = punct_re
        self.negation_re = negation_re
        self.url_re = url_re
        self.rep_char_re = rep_char_re
        self.hashtag_re = hashtag_re
        self.user_tag_re = user_tag_re
        self.acrynoms = self.load_acrynoms()
        self.stemmer = PorterStemmer()

    def load_acrynoms(self):
        with open('./data/acrynom.csv', 'rb') as f:
            reader = csv.reader(f)
            slang = dict((rows[0], rows[1]) for rows in reader)
            return slang
        
    def pos_tags(self, tokens):
        TAG_MAP = [ "NN", "NNP", "NNS", "VBP", "VB", "VBD", 'VBG', "VBN",
                    "VBZ", "MD","UH", "PRP", "PRP$"]
        tags = pos_tag(tokens)
        return [tag[1] for tag in tags if tag[1] in TAG_MAP]

    def normalise(self, tokens):
        
        vect = []
        
        for t in tokens:

            if t in self.stopwords: #t in string.punctuation or t in self.stopwords:
                continue
                
            if not self.emoticon_re.search(t):
                t = t.lower()       
            
            t = self.rep_char_re.sub(r'\1', t)
            t = self.url_re.sub('_URL', t)
            t = self.hashtag_re.sub('_HASH', t)
            t = self.user_tag_re.sub('_USER', t)
            
            vect.append(self.stemmer.stem(t))

        tags = self.pos_tags(tokens)
        vect = tags + vect
        return vect
    
    def tokenise(self, tweet):
        tweet = self.__html2unicode(tweet)
        tokens = self.word_re.findall(tweet)
        return self.normalise(tokens)

    def ensure_unicode(self, tweet):
        try:
            return unicode(tweet)
        except UnicodeDecodeError:
            tweet = str(tweet).encode('string_escape')
            return unicode(tweet)

    def __html2unicode(self, s):
        """
        This function is curtosy of Christopher Potts
        http://sentiment.christopherpotts.net/index.html
        Internal metod that seeks to replace all the HTML entities in
        s with their corresponding unicode characters.
        """
        # First the digits:
        ents = set(self.html_entity_digit_re.findall(s))
        if len(ents) > 0:
            for ent in ents:
                entnum = ent[2:-1]
                try:
                    entnum = int(entnum)
                    s = s.replace(ent, unichr(entnum))
                except:
                    pass
        # Now the alpha versions:
        ents = set(self.html_entity_alpha_re.findall(s))
        ents = filter((lambda x: x != amp), ents)
        for ent in ents:
            entname = ent[1:-1]
            try:
                s = s.replace(ent,
                              unichr(htmlentitydefs.name2codepoint[entname]))
            except:
                pass
            s = s.replace(self.amp, " and ")
        return s

    
"""
    This file is based on the work of Christopher Potts
    however the file has been altered and extended for
    my purposes
    http://sentiment.christopherpotts.net/index.html
"""
emoticon_string = r"""
    (?:
      [<>]?
      [:;=8]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8]                     # eyes
      [<>]?
    )"""

# The components of the tokenizer:
regex_strings = (
    # Phone numbers:
    r""""
    (?:
      (?:            # (international)
        \+?[01]
        [\-\s.]*
      )?
      (?:            # (area code)
        [\(]?
        \d{3}
        [\-\s.\)]*
      )?
      \d{3}          # exchange
      [\-\s.]*
      \d{4}          # base
    )""",
    # Emoticons:
    emoticon_string,
    # HTML tags:
    r'<[^>]+>',
    # Twitter username:
    r'(?:@[\w_]+)',
    # Links
    r'http\S+',
    # Twitter hashtags:
    r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)',
    # Remaining word types:
    r"""
    (?:[a-z][a-z'\-_]+[a-z])       # Words with apostrophes or dashes.
    |
    (?:[+\-]?\d+[,/.:-]\d+[+\-]?)  # Numbers, including fractions, decimals.
    |
    (?:[\w_]+)                    # Words without apostrophes or dashes.
    |
    (?:\.(?:\s*\.){1,})            # Ellipsis dots.
    |
    (?:\S)                         # Everything else that isn't whitespace
    """
    )

negation_words = (
    """
    (?x)(?:
    ^(?:never|no|nothing|nowhere|noone|none|not|
        havent|hasnt|hadnt|cant|couldnt|shouldnt|
        wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint
     )$
    )
    |
    n't
    """
    )

# ######################################################################

word_re = re.compile(r'(%s)' % "|".join(regex_strings), re.VERBOSE | re.I | re.UNICODE)
emoticon_re = re.compile(regex_strings[1], re.VERBOSE | re.I | re.UNICODE)
html_entity_digit_re = re.compile(r'&#\d+;')
html_entity_alpha_re = re.compile(r'&\w+;')
amp = "&amp;"
punct_re = re.compile("^[.:;!?]$")
negation_re = re.compile(negation_words)
url_re = re.compile(r'http\S+')
rep_char_re = re.compile(r'(\w)\1{3,}')
hashtag_re = re.compile(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)')
user_tag_re = re.compile(r'(?:@[\w_]+)')

# Train and evaluate models

In [4]:
import scipy.sparse as sp

def build_and_evaluate(X, y, X_test, y_test, n_gram, min_df, max_df, norm, clf, outpath=None):

    def preprocess(s):
        return preprocessor.tokenise(s)

    # Initialise transformers/estimators
    preprocessor = Preprocessor()
    vec = TfidfVectorizer(tokenizer=preprocess,
                          lowercase=False,
                          ngram_range=n_gram,
                          min_df=min_df,
                          max_df=max_df, 
                          norm=norm)
#                           max_features=5000)
    

    # Build model
    print("Building model")
    tfidf_matrix = vec.fit_transform(X)
    clf.fit(tfidf_matrix, y)

    # Evaluate on test set
    tfidf_matrix_test = vec.transform(X_test)
    y_pred = clf.predict(tfidf_matrix_test)

    print("Classification Report:\n")
    print np.mean(y_pred == y_test)
    print cm(y_test, y_pred)
    print(clsr(y_test, y_pred, target_names=['obj', 'sub']))

# Logistical Regression classifier

In [5]:
clf = LogisticRegression(C=7)
n_gram=(1, 1)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.618705035971
[[82 57]
 [49 90]]
             precision    recall  f1-score   support

        obj       0.63      0.59      0.61       139
        sub       0.61      0.65      0.63       139

avg / total       0.62      0.62      0.62       278



# Naive Bayes

In [6]:
clf = MultinomialNB()
n_gram = (1, 1)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram , 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.676258992806
[[ 85  54]
 [ 36 103]]
             precision    recall  f1-score   support

        obj       0.70      0.61      0.65       139
        sub       0.66      0.74      0.70       139

avg / total       0.68      0.68      0.67       278



# Linear SVC classifier

In [7]:
clf = LinearSVC(C=5)#, penalty='l1', dual=False)
n_gram=(1, 2)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.593525179856
[[92 47]
 [66 73]]
             precision    recall  f1-score   support

        obj       0.58      0.66      0.62       139
        sub       0.61      0.53      0.56       139

avg / total       0.60      0.59      0.59       278



# SGD Classifier

In [8]:
clf = SGDClassifier()
n_gram=(1, 2)
model = build_and_evaluate(train['text'].values, train['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.593525179856
[[92 47]
 [66 73]]
             precision    recall  f1-score   support

        obj       0.58      0.66      0.62       139
        sub       0.61      0.53      0.56       139

avg / total       0.60      0.59      0.59       278



# Logistical Regression classifier large training set

In [9]:
clf = LogisticRegression(C=7)
n_gram=(1, 2)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.654676258993
[[ 67  72]
 [ 24 115]]
             precision    recall  f1-score   support

        obj       0.74      0.48      0.58       139
        sub       0.61      0.83      0.71       139

avg / total       0.68      0.65      0.64       278



# Naive Bayes large training set

In [10]:
clf = MultinomialNB()
n_gram = (1, 1)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram , 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.625899280576
[[ 59  80]
 [ 24 115]]
             precision    recall  f1-score   support

        obj       0.71      0.42      0.53       139
        sub       0.59      0.83      0.69       139

avg / total       0.65      0.63      0.61       278



# Linear SVC classifier large training set

In [11]:
clf = LinearSVC(C=5)#, penalty='l1', dual=False)
n_gram=(1, 2)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.661870503597
[[ 64  75]
 [ 19 120]]
             precision    recall  f1-score   support

        obj       0.77      0.46      0.58       139
        sub       0.62      0.86      0.72       139

avg / total       0.69      0.66      0.65       278



# SGD Classifier large training set

In [12]:
clf = SGDClassifier()
n_gram=(1, 2)
model = build_and_evaluate(train_large['text'].values, train_large['class'].values,
                           test['text'].values, test['class'].values,
                           n_gram, 1, 0.8, 'l2', clf)

Building model
Classification Report:

0.665467625899
[[ 71  68]
 [ 25 114]]
             precision    recall  f1-score   support

        obj       0.74      0.51      0.60       139
        sub       0.63      0.82      0.71       139

avg / total       0.68      0.67      0.66       278

